## 筛选
filter(cond):筛选出满足条件cond的行。cond可以填字符串，格式和SQL中的where子句一样，也可以填Bool类型的Column对象，比如 df['a']>1。

In [26]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
t1 = df.filter("a > 1 and c < 9")
t1.show() # 输出 4,5,6 这一行
t2 = df.filter( (df['b']<5) & (df['c']<8)) # 可以使用&或|对两个bool列进行逻辑运算，但必须要用圆括号括起，限定运算顺序。
t2.show() # 输出 1,2,3 这一行
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  4|  5|  6|
+---+---+---+

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
+---+---+---+



## 赋值，加列
withColumn(col_name,col):col_name是列名，col是列值，必须是一个Column对象。
赋值和加列操作是相同的，col_name存在，就是赋值，否则就是加列。

In [27]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
t1 = df.withColumn("c",df['c']+1)
t1.show() # c的值全都增加了1
t2 = df.withColumn("d",df['a']+1)
t2.show() # 增加了新一列d
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  4|
|  4|  5|  7|
|  7|  8| 10|
+---+---+---+

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  2|
|  4|  5|  6|  5|
|  7|  8|  9|  8|
+---+---+---+---+



## 删除列
drop(col_name)：col_name为列名。该方法会返回一个删除col_name列的DataFrame

In [30]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
df.show()
t = df.drop("c")
t.show() # 只有 a,b两列
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+

+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  4|  5|
|  7|  8|
+---+---+



## 给列取名
alias(col_name):通常和select配合使用，请看下面的例子

## 选取列顺序
select(*cols):cols为列名或列对象。
赋值和删除操作，每次只能改加减一列数据，如果想要批量地改变，尤其是调整列顺序的时候，就非常有用了。在ETL中，当需要计算的列很多时，通常就是逐个计算出不同的列对象，最后用select把它们排好顺序。



In [34]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
df.show()
a1 = (df['a']+1).alias("a+1") # 新增一个列对象，取名为a1
t = df.select("a",a1,"b") # 如果用字符串，必须是df中存在的列名。
t.show() # 显示a, a_1,b 三列
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+

+---+---+---+
|  a|a+1|  b|
+---+---+---+
|  1|  2|  2|
|  4|  5|  5|
|  7|  8|  8|
+---+---+---+



## 加入特殊Column对象
在赋值的例子里，Column对象是由原DataFrame的Column经过简单的数学运算或逻辑运算得到的，但如果我们想生成一些更特殊的Column呢？比如常数列或者自己定义复杂的规则。
Spark提供了pyspark.sql.functions,含有丰富的接口，其中就有我们需要的东西。篇幅有限，只能介绍一些常用的，更多的还是需要去看官方文档。

常数列
lit(value):value数必须是必须为pyspark.sql.types支持的类型，比如int,double,string,datetime等



In [38]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import lit
from datetime import datetime
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
df.show()
t = df.withColumn("some_datetime",lit(datetime(2018,1,1,2,3,4,999)))
t.show(truncate=False)
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+

+---+---+---+--------------------------+
|a  |b  |c  |some_datetime             |
+---+---+---+--------------------------+
|1  |2  |3  |2018-01-01 02:03:04.000999|
|4  |5  |6  |2018-01-01 02:03:04.000999|
|7  |8  |9  |2018-01-01 02:03:04.000999|
+---+---+---+--------------------------+



## 取整
round、floor：和Python的标准函数用法一致，只是数字换成列名

## 条件分支
when(cond,value):符合cond就取value，value可以是常数也可以是一个列对象，连续可以接when构成多分支
otherwise(value):接在when后使用，所有不满足when的行都会取value,若不接这一项，则取Null。

In [40]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import when
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
df.show()
t = df.withColumn("when",when(df['a']==1,"a=1").when(df['b']==5,df['b']%5).otherwise("other"))
t.show() # 生成when列，值分别为 a=1,0,other
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+

+---+---+---+-----+
|  a|  b|  c| when|
+---+---+---+-----+
|  1|  2|  3|  a=1|
|  4|  5|  6|    0|
|  7|  8|  9|other|
+---+---+---+-----+



## 日期和时间
- current_date():当前日期，返回一个date列
- current_timestamp():当前时刻，返回一个timestamp列
- date_add(start, days)：日期正向偏移，start为开始时间，必须是Column或字符串对象，指向一个date或timestamp列，days为偏移天数。
- date_sub(start, days)：类似date_add,但是负向偏移。
- date_format(date, format): 日期格式化，date为要格式化的时间，必须是Column或字符串对象，指向一个date或timestamp列，days为偏移天数，format为格式化的字符串，具体参考Hive QL的date_format函数。
- datediff(end, start)：计算天数差

## 自定义规则
**udf(f, returnType=StringType): 用户自定义处理函数**。
f为自定义的处理函数，returnType为f的返回类型，必须为pyspark.sql.types支持的类型，如果不填，会默认自动转化为String类型。udf会返回一个函数，可以当做列函数使用。

这在处理逻辑非常复杂时很有用。比如对身份证号进行校验计算，然后取出有效的身份证号的第1，4，10位，这个复杂流程很难用Spark提供的API拼接起来，只能自己写。

自定义函数f的传入参数为列的值。

In [41]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,2,3],[4,5,6],[7,8,9]],['a','b','c'])
df.show()
def f(a,b,c):
    r=0
    if a==1:
        r=1
    elif b==5:
        r=2
    return r

col_match = udf(f,IntegerType())
t = df.withColumn("col_match",col_match("a","b","c"))
t.show() # 生成col_match列，值分别为 a=1,2,0
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+

+---+---+---+---------+
|  a|  b|  c|col_match|
+---+---+---+---------+
|  1|  2|  3|        1|
|  4|  5|  6|        2|
|  7|  8|  9|        0|
+---+---+---+---------+



## 排序
Spark支持多字段，升降序排序。
可以使用orderBy和sort，因为操作比较简单也符合直觉，这里略去例子，详情可以看文档。

## 聚合
Spark 支持直接聚合，也支持分组聚合。聚合的表达方式非常多，这里仅选取常用的。

### 直接聚合

In [69]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import sum
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,1,3],[4,1,2],[7,2,9]],['a','b','c'])
df.show()
t = df.agg(sum("a"))
print(t.collect()[0][0]) # 打印 12
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  1|  3|
|  4|  1|  2|
|  7|  2|  9|
+---+---+---+

12


### 分组聚合

In [68]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import sum,max
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
df = hc.createDataFrame([[1,1,3],[4,1,2],[7,2,9]],['a','b','c'])
df.show()
t = df.groupBy("b").agg(sum("a"),max("c"))
t.show() 
sc.stop()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  1|  3|
|  4|  1|  2|
|  7|  2|  9|
+---+---+---+

+---+------+------+
|  b|sum(a)|max(c)|
+---+------+------+
|  1|     5|     3|
|  2|     7|     9|
+---+------+------+



## 窗口函数
有一类分析需求，是需要分组计算，但保持数据的粒度不变的。比如通过成绩表，按班计算的学生的成绩排名，加一列到原本的成绩表中，整个表的每一行仍然表示一名学生。这种分析需求称为窗口分析，比如说每个班，就是一个窗口，在这个窗口中，计算出班级成绩排名，再并到原表中。
这种分析，首先要创建一个窗口，然后再使用窗口函数来进行计算。Spark提供了丰富的窗口函数，可以满足各类分析需求。

### 创建窗口
使用pyspark.sql.Window对象可以创建一个窗口，最简单的窗口可以什么都没有，但一般不推荐这样做。可以使用partitionBy进行分组，使用orderBy进行排序，比如
```
from pyspark.sql import Window
window = Window.partitionBy("a").orderBy("b")

```

### 窗口函数使用示例
rank()：根据窗口中partitionBy进行分组，以orderBy排序

下面示例：按班级，分数从高到低，生成排名



In [76]:
# -*- coding: UTF-8 -*-
from pyspark import SparkContext, HiveContext
from pyspark.sql.functions import rank,desc, row_number
from pyspark.sql import Window
sc = SparkContext(appName="transform")
hc = HiveContext(sc)
score = [
    ['a','a_1',90],
    ['a','a_2',80],
    ['a','a_3',85],
    ['b','b_1',70],
    ['b','b_2',80],
    ['b','b_3',75],
    ['c','c_1',90]
]
df = hc.createDataFrame(score,['class','student','score'])
class_window = Window.partitionBy("class").orderBy(desc("score")) #降序排列
df.show()
class_rank = rank().over(class_window) 
class_row_number = row_number().over(class_window) #窗口函数(xxx).over(window)，就是一般的用法
t = df.withColumn("rank",class_rank)
t = df.withColumn("row_number",class_row_number)
t.show() 
sc.stop()

+-----+-------+-----+
|class|student|score|
+-----+-------+-----+
|    a|    a_1|   90|
|    a|    a_2|   80|
|    a|    a_3|   85|
|    b|    b_1|   70|
|    b|    b_2|   80|
|    b|    b_3|   75|
|    c|    c_1|   90|
+-----+-------+-----+

+-----+-------+-----+----------+
|class|student|score|row_number|
+-----+-------+-----+----------+
|    c|    c_1|   90|         1|
|    b|    b_2|   80|         1|
|    b|    b_3|   75|         2|
|    b|    b_1|   70|         3|
|    a|    a_1|   90|         1|
|    a|    a_3|   85|         2|
|    a|    a_2|   80|         3|
+-----+-------+-----+----------+



## 缓存
在实际业务中，常常会碰到这种需求：需要把一个计算结果，稍加不同的改动，分别存为不同的表。比如，ETL中为了保证出错后能重试，就会要求除了保存转换计算结果之外，还要备份一份到备份表里。备份表通常是按天分区的，每个区存当天的转换计算结果。而应用表则不分区，只存最新一天的计算结果。

在完成这一需求时，如果是先保存应用表，然后再添加分区列后添加到分区表，就会触发两次完整的计算流程，时间翻倍。而如果有缓存，就不一样了。我们可以在计算到最终结果时，缓存一下这张表，然后把它保存为应用表，再添加分区列保存为分区表。那么，实际计算中，到缓存操作为止的计算，只会触发一次，实际消耗时间为1次到最终结果的计算+1次加分区列，远小于2次计算的时间。

当某些中间结果需要反复使用时，缓存可以给我们带来极大的效率提升。当然，相应地，内存也会占用更多，还是应该根据具体情况决定如何取舍。缓存的方法很简单，只要让DataFrame对象执行cache方法就行了：`df.cache()`